In [1]:
# General imports
import numpy as np
import torch
import matplotlib.pyplot as plt
# DeepMoD stuff
from deepymod import DeepMoD
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold, PDEFIND
from deepymod.training import train
from deepymod.training.sparsity_scheduler import TrainTestPeriodic
from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta

#if torch.cuda.is_available():
#    device = 'cuda'

#else:
device = 'cpu'

# Settings for reproducibility
np.random.seed(44)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Making dataset

noise = 0.2
A = 1
v = 0.25

In [2]:
runs = 1
dataset = Dataset(BurgersDelta, A=A, v=v)

In [ ]:
#for n_x in np.array([4,6,8,10,12,14,16,20,25,30]):
for n_x in np.array([18,22,35]):
    for run in np.arange(runs):
        
        x = np.linspace(-4, 4, n_x) 
        t = np.linspace(0.1, 1.1, 100) 
        t_grid, x_grid = np.meshgrid(t, x, indexing='ij')
        X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=0, noise=noise, random=True, normalize=False)
        
        X, y = X.to(device), y.to(device)
        network = NN(2, [30, 30, 30, 30], 1)
        library = Library1D(poly_order=2, diff_order=3) # Library function
        estimator = Threshold(0.2) # Sparse estimator 
        constraint = LeastSquares() # How to constrain
        model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=500, delta=1e-7) # in terms of write iterations
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) # Defining optimizer

        train(model, X, y, optimizer, sparsity_scheduler, log_dir='runs/normal_grid_noise_20/normal_grid'+str(n_x), split=0.8, write_iterations=25, max_iterations=100000, delta=1e-4, patience=500) 
        print(model.constraint_coeffs(sparse=True, scaled=True))

 99975  MSE: 1.84e-04  Reg: 3.39e-08  L1: 2.16e+00 Algorithm converged. Writing model to disk.
[tensor([[ 0.0000],
        [ 0.0000],
        [ 0.7506],
        [ 0.0000],
        [ 0.0000],
        [-1.4080],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]], grad_fn=<MulBackward0>)]
 14050  MSE: 1.15e-03  Reg: 6.42e-06  L1: 2.07e+00 Algorithm converged. Writing model to disk.
[tensor([[ 0.0000],
        [ 0.0000],
        [ 0.9112],
        [ 0.0000],
        [ 0.0000],
        [-0.6642],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [-0.4917],
        [ 0.0000],
        [ 0.0000]], grad_fn=<MulBackward0>)]
 99975  MSE: 1.70e-03  Reg: 6.75e-07  L1: 4.36e+00 Algorithm converged. Writing model to disk.
[tensor([[ 0.0000],
        [-1.3806],
        [ 0.7080],
        [ 0.6758],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 1.5970],
        [ 0.00

In [ ]:
# General imports
import numpy as np
import torch
import matplotlib.pyplot as plt
# DeepMoD stuff
from deepymod import DeepMoD
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold, PDEFIND
from deepymod.training import train
from deepymod.training.sparsity_scheduler import TrainTestPeriodic
from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta

#if torch.cuda.is_available():
#    device = 'cuda'

#else:
device = 'cpu'

# Settings for reproducibility
np.random.seed(44)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Making dataset

noise = 0.01
A = 1
v = 0.25

runs = 1
dataset = Dataset(BurgersDelta, A=A, v=v)

for n_x in np.array([16,18,22,35,20,25,30]):
#for n_x in np.array([2]):
    for run in np.arange(runs):
        
        x = np.linspace(-4, 4, n_x) 
        t = np.linspace(0.1, 1.1, 100) 
        t_grid, x_grid = np.meshgrid(t, x, indexing='ij')
        X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=0, noise=noise, random=True, normalize=False)
        X, y = X.to(device), y.to(device)
        network = NN(2, [30, 30, 30, 30], 1)
        library = Library1D(poly_order=2, diff_order=3) # Library function
        estimator = Threshold(0.2) # Sparse estimator 
        constraint = LeastSquares() # How to constrain
        model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=500, delta=1e-7) # in terms of write iterations
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) # Defining optimizer

        train(model, X, y, optimizer, sparsity_scheduler, log_dir='runs/normal_grid/normal_grid'+str(n_x), split=0.8, write_iterations=25, max_iterations=100000, delta=1e-4, patience=500) 
        print(model.constraint_coeffs(sparse=True, scaled=True))
        